Dementia prediction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dataset import 😁😁😁😁😁


In [18]:
df = pd.read_csv("dementia_dataset.csv")

Dataset Clean part

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Subject ID  373 non-null    object 
 1   MRI ID      373 non-null    object 
 2   Group       373 non-null    object 
 3   Visit       373 non-null    int64  
 4   MR Delay    373 non-null    int64  
 5   M/F         373 non-null    object 
 6   Hand        373 non-null    object 
 7   Age         373 non-null    int64  
 8   EDUC        373 non-null    int64  
 9   SES         354 non-null    float64
 10  MMSE        371 non-null    float64
 11  CDR         373 non-null    float64
 12  eTIV        373 non-null    int64  
 13  nWBV        373 non-null    float64
 14  ASF         373 non-null    float64
dtypes: float64(5), int64(5), object(5)
memory usage: 43.8+ KB


In [21]:
df.describe()

,Visit,MR Delay,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
count,373.000000,373.000000,373.000000,373.000000,354.000000,371.000000,373.000000,373.000000,373.000000,373.000000
mean,1.882038,595.104558,77.013405,14.597855,2.460452,27.342318,0.290885,1488.128686,0.729568,1.195461
std,0.922843,635.485118,7.640957,2.876339,1.134005,3.683244,0.374557,176.139286,0.037135,0.138092
min,1.000000,0.000000,60.000000,6.000000,1.000000,4.000000,0.000000,1106.000000,0.644000,0.876000
25%,1.000000,0.000000,71.000000,12.000000,2.000000,27.000000,0.000000,1357.000000,0.700000,1.099000
50%,2.000000,552.000000,77.000000,15.000000,2.000000,29.000000,0.000000,1470.000000,0.729000,1.194000
75%,2.000000,873.000000,82.000000,16.000000,3.000000,30.000000,0.500000,1597.000000,0.756000,1.293000
max,5.000000,2639.000000,98.000000,23.000000,5.000000,30.000000,2.000000,2004.000000,0.837000,1.587000


In [5]:
df.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [27]:
df = df.drop(columns=["Subject ID", "MRI ID"])

df.head()

KeyError: "['Subject ID', 'MRI ID'] not found in axis"

In [28]:
df.rename(columns={"M/F": "Gender"}, inplace=True)
df.columns

Index(['Group', 'Visit', 'MR Delay', 'Gender', 'Age', 'EDUC', 'SES', 'MMSE',
       'CDR', 'eTIV', 'nWBV', 'ASF'],
      dtype='object')

In [29]:
df.isnull().sum()

Group        0
Visit        0
MR Delay     0
Gender       0
Age          0
EDUC         0
SES         19
MMSE         2
CDR          0
eTIV         0
nWBV         0
ASF          0
dtype: int64

In [30]:
# df["SES"] = df["SES"].fillna(0)
df["SES"] = df["SES"].bfill()

In [31]:
df["MMSE"] = df["MMSE"].bfill()

In [32]:
df['Gender'] = df['Gender'].map({'M': 0, 'F': 1})

In [33]:
df.head()

,Group,Visit,MR Delay,Gender,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,Nondemented,1,0,0,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,Nondemented,2,457,0,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,Demented,1,0,0,75,12,3.0,23.0,0.5,1678,0.736,1.046
3,Demented,2,560,0,76,12,3.0,28.0,0.5,1738,0.713,1.010
4,Demented,3,1895,0,80,12,3.0,22.0,0.5,1698,0.701,1.034


Model Train

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [35]:
x = df.iloc[:,1:]
y = df.iloc[:,0]

In [36]:
y

0      Nondemented
1      Nondemented
2         Demented
3         Demented
4         Demented
          ...     
368       Demented
369       Demented
370    Nondemented
371    Nondemented
372    Nondemented
Name: Group, Length: 373, dtype: object

In [37]:
# df['SES'] = df['SES'].apply(lambda x: 0 if str(x).startswith('<bound method') else x)
# df['SES'] = pd.to_numeric(df['SES'], errors='coerce').fillna(0)

In [38]:
model = RandomForestClassifier(n_estimators=100)
model.fit(x,y)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [39]:
df1 = pd.read_csv("C:\\Users\\Hp\\Desktop\\college project\\Early-Stage-Dementia-Detection\\model traning\\synthetic_mri_data.csv")

In [40]:
x_test = df1.iloc[:,:-1]
y_test = df1.iloc[:,-1]

In [41]:
x_test

,MR Delay,Gender,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,0,0,63,10,0.0,27.6,0.0,1984,0.842,1.231
1,1187,0,66,17,1.0,26.8,1.0,1373,0.684,1.207
2,0,0,64,15,1.0,26.5,0.5,1640,0.736,1.089
3,942,0,75,14,1.0,29.4,0.0,2071,0.826,1.164
4,1835,0,76,20,3.0,26.9,1.0,1390,0.741,0.981
...,...,...,...,...,...,...,...,...,...,...
95,988,0,81,16,1.0,28.2,0.0,2062,0.707,1.307
96,146,0,77,14,2.0,24.9,1.0,1318,0.734,0.944
97,1824,0,68,11,2.0,29.2,0.0,1524,0.805,1.342
98,1383,1,61,16,2.0,29.4,0.0,1585,0.838,1.313


In [43]:
y_pred = model.predict(x_test)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Visit


In [ ]:

accuracy = round(accuracy_score(y_pred, y_test) * 100, 2)

In [ ]:
print(accuracy)

100.0


In [ ]:
print(classification_report(y_test,y_pred ))

              precision    recall  f1-score   support

    Demented       1.00      1.00      1.00        46
 Nondemented       1.00      1.00      1.00        54

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [ ]:
from sklearn.model_selection import KFold, cross_val_score
#declare the k size
kf = KFold(n_splits=10, shuffle=True, random_state=42) # 5 folds, shuffled, for reproducibility

# Perform cross-validation
scores = cross_val_score(model, x, y, cv=kf, scoring='accuracy')
print(scores)

NameError: name 'model' is not defined

In [44]:
import joblib
joblib.dump(model, "dementia_rf_model.joblib")


['dementia_rf_model.joblib']